In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image, ImageDraw
import os
import pandas as pd
import tqdm
from src.model import FaceVerification
from torchvision.transforms import v2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
transforms = v2.Compose([
    v2.RandomHorizontalFlip(),
    # v2.RandomRotation(5),
    # v2.RandomResizedCrop(160, scale=(0.8, 1.0)),
    v2.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    v2.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    v2.RandomGrayscale(p=0.1),
    v2.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    v2.Resize((512, 512)),
    v2.ToTensor(),
    v2.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
])

In [7]:
from torchvision import datasets

dataset = datasets.ImageFolder('Face Dataset/Train', transform=transforms)
dataloader = DataLoader(dataset, shuffle=True)
num_classes = len(dataset.classes)


In [8]:
model = FaceVerification(num_classes=num_classes, train_backbone=True)
model.train(dataloader, num_epochs=100)

torch.Size([1, 3, 512, 512])


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512])

In [7]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import cv2
from IPython.display import display
import matplotlib.pyplot as plt

mtcnn = MTCNN(keep_all=True)
resnet = InceptionResnetV1(pretrained='vggface2').eval()

img = cv2.imread('/Users/tranthanh/Documents/Deep Learning/Face-Verification/Face Dataset/Train/NgonRuoi/NgonRuoi.jpeg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(img.dtype)
img = Image.fromarray(img)
print(img.size)
# boxes, probs = mtcnn.detect(img)
#Get face tensor
face_tensors = mtcnn(img)
face_tensors = transforms(face_tensors)
print(face_tensors.shape)
# plt.imshow(face_tensors[0].permute(1, 2, 0))

uint8
(2320, 3088)
torch.Size([1, 3, 160, 160])
